In [1]:
# imports
import sys
sys.path.append('..')
import keras
import numpy as np
import tensorflow as tf
from keras.models import load_model
import pandas as pd
import os
import pickle

from keras.backend.tensorflow_backend import set_session
from BioExp.helpers.metrics import *
from BioExp.helpers.losses import *

Using TensorFlow backend.


In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[]

In [3]:
# GPU setup
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

In [5]:
# model and parameter defn
seq_map = {'flair': 0, 't1': 1, 't2': 3, 't1c':2}
seq = 'flair'

model_path        = '../saved_models/model_{}_scaled/model-archi.h5'.format(seq)
weights_path      = '../saved_models/model_{}_scaled/model-wts-{}.hdf5'.format(seq, seq)

layers_to_consider = ['conv2d_2', 'conv2d_3', 'conv2d_5', 'conv2d_7','conv2d_9',\
                      'conv2d_11', 'conv2d_13', 'conv2d_15', 'conv2d_17', 'conv2d_19', 'conv2d_21']

model = load_model(model_path, custom_objects={'gen_dice_loss':gen_dice_loss,
                                'dice_whole_metric':dice_whole_metric,
                                'dice_core_metric':dice_core_metric,
                                'dice_en_metric':dice_en_metric})
model.load_weights(weights_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
model.summary()

## Causal Inference method
![pipeline](../imgs/causal_pipeline.png)

### Concept Generation

We make use of various standard clustering techniques to cluster network weights into various groups. This individual cluster group is considered as a concept. The rationale behind doing this is, we consider that the weights responsible for similar tasks form a cluster; for example, all the weights responsible for edges will form one cluster, while the weights responsible for corners form other clusters.

For faster inference, we make layer selection as a hyperparameter, i.e we don't run inference on the entire network but select some layers and construct the causal links among the neurons between them. 


In [ ]:
from BioExp.clusters import clusters

concept_info = []
node = 0

save_root = './Logs/weights_cluster/'
for layer_name in layers_to_consider:
    save_path = os.path.join(save_root, layer_name)
    os.makedirs(save_path, exist_ok = True)
    
    C = clusters.Cluster(model, weights_path, layer_name)
    labels = C.get_clusters(threshold = 0.5, save_path=save_path)
    C.plot_weights(labels, 5, os.path.join(save_path, 'wt-samples'))
    
    for label in np.unique(labels):
        nodename = 'node_{}'.format(node)
        layername = layer_name
        fidxs = np.where(labels==label)[0]
        info = {'concept_name': nodename, 
                  'layer_name': layername, 
                 'filter_idxs': fidxs}
        concept_info.append(info)
        node += 1
        
with open(os.path.join(save_path, 'cluster_info.cpickle'), 'wb') as file:
    pickle.dump(concept_info, file)

### TODO

Another proposed method for clustering is to hand-engineered features using statistical methods. We carefully select features responsible for orientations, intensity, and textures using statistical tricks, which are further used by GMM's for grouping them together. This method still needs some thoughts to make it work as expected, which will be explored in future works.

- [ ] Different clustering methods by hand engineering features

### Concept Identification

We use gradient based method to visualize and identify concepts

+ Generated multinomial distribution can be used as a proxy for each concepts
+ To verify that gaussian is not spread over, and has some nice range

Flow Identifier prints dice of the binarized concept with ground truth and also tries to estimate entropy

For concept identification, we use a gradient-based method to find the activated region on the input image, indicating the effective concepts in an input image. Mathematically the same is indicated as described in equation $Concept = \mathbb{E}(\nabla_{\phi} I)$ 

Once the clusters are obtained, we conduct a significance analysis for each cluster group to make sure the cluster selected is statistically significant in selecting the features. To accomplish the same, we first generate the distribution for each concept, which is done by assuming Multidimensional-Gaussian Prior, whose parameters are estimated using all the elements in the clusters. Since the concepts being ergodic, we can consider sample statistics for a probability distribution.

Once when we have the distribution, we sample several concepts and run concept identification test to map the similarities between original and sampled concepts. One with higher variance with respect to an original concept is removed completely from further tests, as it doesn't coherently map to any concepts in an input image.

In [ ]:
from BioExp.clusters.concept import ConceptIdentification
from BioExp.helpers import utils

image, gt = utils.load_vol_brats('../sample_vol/brats/Brats18_CBICA_AOP_1', slicen=105)
image = image[:, :, seq_map[seq]][:,:, None]

save_root = './Logs/weights_cluster/'
with open(os.path.join(save_root, 'cluster_info.cpickle'), 'rb') as file:
    concepts_info = pickle.load(file)
    
metric = dice_label_coef # defined in helpers.losses
identifier = ConceptIdentification(model, weights_path, metric)

save_root = './Logs/concept_identification'


for concept_info in concepts_info:        
    identifier.flow_based_identifier(concept_info, 
                           save_path = os.path.join(save_root, 
                                                    concept_info['layer_name']), 
                           test_img = image,
                           test_gt = gt)
    
    identifier.check_robustness(concept_info,
                            save_path = os.path.join(save_root, 
                                                     concept_info['layer_name']), 
                            test_img = image,
                            test_gt = gt,
                            save_all = True,
                            nmontecarlo = 4)

### Update concept info manual inspection

+ Remove all the concepts with blank or features all over the place
+ Need to define some metrics to eleminate useless concepts

In [ ]:
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt

save_root = './Logs/concept_identification'
paths = np.array([pth for pth in glob(save_root + '/*/*.png') if pth.__contains__('robustness')])
for pth in paths:
    print ("========={}========".format(pth))
    plt.imshow(np.array(Image.open(pth)))
    plt.show()


In [ ]:
from pprint import pprint
consider_nodes = [{'node': 26, 'description': 'Left Lower Brain boundary'}, 
                  {'node': 27, 'description': 'Left Brain boundary'}, 
                  {'node': 28, 'description': 'Right Brain boundary'},
                  {'node': 29, 'description': 'Upper Brain boundary'},
                  {'node': 31, 'description': 'Inner brain and tumor boundary'},
                  {'node': 32, 'description': 'Inner brain and brain boundary'},
                  {'node': 33, 'description': 'Inner brain'},
                  {'node': 34, 'description': 'Inner brain dense structure'},
                  {'node': 36, 'description': 'Tumor Core'},
                  {'node': 37, 'description': 'Brain boundary and tumor core'},
                  {'node': 38, 'description': 'Tumor Core'},
                  {'node': 39, 'description': 'Inner Brain Boundary'},
                  {'node': 40, 'description': 'Tumor Boundary'},
                  {'node': 41, 'description': 'Tumor Core and tuomr boundary'},
                  {'node': 42, 'description': 'Left brain and tumor core'},
                  {'node': 43, 'description': 'Thick tumor boundary'},
                  {'node': 44, 'description': 'Tumor regions'}]

# update cluster info
save_root = './Logs/weights_cluster/'
with open(os.path.join(save_root, 'cluster_info.cpickle'), 'rb') as file:
    concepts_info = pickle.load(file)
    
modified_concepts = []
for concept_info in concepts_info:
    for i, node in enumerate(consider_nodes):
        if node['node'] == int(concept_info['concept_name'].split('_').pop()):
            concept_info['description'] = consider_nodes[i]['description']
            pprint(concept_info)
            modified_concepts.append(concept_info)
            break

with open('./Logs/modified_clusters.cpickle', 'wb') as file:
    pickle.dump(modified_concepts, file)  

### Causal Graph Links

Based on information flow and divergance properties, still the network in correlation but the constraints imposed pushes correlation towards causality.

After the estimation of a significant concept, the next step is to construct the graph using these concepts, which kind captures causal interactions between the concepts. To construct this graph, we work with feature map distributions; mathematically, the link between any two concept nodes exists only if KL divergence is less than a threshold value, which is described in bellow equations.

Let, $C_i^l$ denotes the $i^{th}$ concept in layer $l$. The directed link $C_i^p \rightarrow C_j^q$ is determined by $KL(\mathbb{Q}(x~|~\Phi_{j,q})~||~ \mathbb{Q}(x~|~\Phi_{q}))$, where $\mathbb{Q}$ is the distribution of feature maps obtained by considering network $\Phi$. $\mathbb{Q}(x~|~\Phi_{j,q})$ is the distribution of feature maps generated as a result of considering just single concept $C_j^q$, while $\mathbb{Q}(x~|~\Phi_{q})$ is the distribution generated by considering all the concepts in layer $q$ and setting all $C_{-i}^p$ to zeros.

The directed link $C_i^p \rightarrow C_j^q$, exists only if $KL(\mathbb{Q}(x~|~\Phi_{j,q})~||~ \mathbb{Q}(x~|~\Phi_{q})) < T$ where T is the threshold parameter.


In [ ]:
from BioExp.graphs.causal import CausalGraph
from BioExp.helpers import utils

metric = dice_label_coef # defined in helpers.losses
CG = CausalGraph(model, weights_path, metric)


with open('./Logs/modified_clusters.cpickle', 'rb') as file:
    concept_info = pickle.load(file)

dataset_path = '../sample_vol/brats/'
def dataloader(nslice = 78):
    def loader(img_path):
        image, gt =  utils.load_vol_brats(img_path, slicen=nslice)
        return image[:, :, seq_map[seq]][:,:, None], gt
    return loader

save_path = './Logs/Graphs/causal'
os.makedirs(save_path, exist_ok=True)
CG.generate_graph(concept_info, 
                  dataset_path, 
                  dataloader(), 
                  edge_threshold = 0.5, 
                  save_path = save_path, 
                  max_samples=10)

In [8]:
import pickle

graph_path = './Logs/Graphs/causal/causal_graph.pickle'
with open(graph_path, 'rb') as file:
    graph_json = pickle.load(file)

root_node = graph_json['rootNode']
graph = graph_json['graph']

In [9]:
graph.print(root_node)

node: Input, children: ['node_26', 'node_27', 'node_28', 'node_29'], parents: []
node: node_26, children: ['node_31', 'node_32', 'node_37', 'node_39', 'node_40', 'node_42'], parents: ['Input']
node: node_27, children: ['node_31', 'node_32', 'node_33', 'node_37', 'node_38', 'node_39', 'node_40', 'node_42', 'node_43'], parents: ['Input']
node: node_28, children: ['node_31', 'node_32', 'node_37', 'node_38', 'node_39', 'node_40', 'node_42'], parents: ['Input']
node: node_29, children: ['node_32', 'node_37', 'node_39', 'node_40', 'node_41', 'node_42'], parents: ['Input']
node: node_31, children: ['node_37', 'node_38', 'node_39', 'node_40', 'node_41', 'node_42'], parents: ['node_26', 'node_27', 'node_28']
node: node_32, children: ['node_37', 'node_38', 'node_39', 'node_40', 'node_42'], parents: ['node_26', 'node_27', 'node_28', 'node_29']
node: node_37, children: ['node_40', 'node_41', 'node_42'], parents: ['node_26', 'node_27', 'node_28', 'node_29', 'node_31', 'node_32', 'node_33']
node: no

In [10]:
from pgm.helpers.trails import findTrails

ftrails = findTrails(root_node, 'Input', 'node_41')

with open('./Logs/modified_clusters.cpickle', 'rb') as file:
    concepts = pickle.load(file)

trails = []
trailsdescription = []

for ntrail in ftrails.trails:
    trail = ['Input']
    traildescription = ['Input Image']
    for node in ntrail:
        for concept in concepts:
            if node.name == concept['concept_name']:
                trail.append(node.name)
                traildescription.append(concept['description'])
                break
    trails.append(trail)
    trailsdescription.append(traildescription)
    #print (trail)
    print (traildescription)

['Input Image', 'Left Lower Brain boundary', 'Inner brain and tumor boundary', 'Brain boundary and tumor core', 'Tumor Core and tuomr boundary']
['Input Image', 'Left Lower Brain boundary', 'Inner brain and tumor boundary', 'Tumor Core', 'Tumor Core and tuomr boundary']
['Input Image', 'Left Lower Brain boundary', 'Inner brain and tumor boundary', 'Tumor Core and tuomr boundary']
['Input Image', 'Left Lower Brain boundary', 'Inner brain and brain boundary', 'Brain boundary and tumor core', 'Tumor Core and tuomr boundary']
['Input Image', 'Left Lower Brain boundary', 'Inner brain and brain boundary', 'Tumor Core', 'Tumor Core and tuomr boundary']
['Input Image', 'Left Lower Brain boundary', 'Brain boundary and tumor core', 'Tumor Core and tuomr boundary']
['Input Image', 'Left Brain boundary', 'Inner brain and tumor boundary', 'Brain boundary and tumor core', 'Tumor Core and tuomr boundary']
['Input Image', 'Left Brain boundary', 'Inner brain and tumor boundary', 'Tumor Core', 'Tumor Co

In [11]:
n = graph.get_node('node_41')